In [ ]:
! pip install -U sentence-transformers
! pip install -q wandb py_vncorenlp

import py_vncorenlp
py_vncorenlp.download_model(save_dir='/kaggle/working/')
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/default-java'
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/kaggle/working')

In [4]:
import pandas as pd
import numpy as np
import torch
import pickle
import math
import json
from datasets import load_dataset, DatasetDict, Dataset
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, models, losses, util, datasets
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from sentence_transformers.cross_encoder.evaluation import CESoftmaxAccuracyEvaluator
from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)
import numpy as np

2024-06-23 05:28:30.598592: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-23 05:28:30.598708: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-23 05:28:30.726635: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
no_segment_flatten_corpus = []
with open("/kaggle/input/law-qa/flatten_law_corpus.json", "r") as file:
    for line in file:
        data = json.loads(line)
        no_segment_flatten_corpus.append(data)

In [6]:
no_segment_queries = []

with open("/kaggle/input/law-qa/query_set_evaluate.json", "r") as file:
    for line in file:
        data = json.loads(line)
        no_segment_queries.append(data)

In [7]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(no_segment_queries, test_size=0.2, random_state=42)

In [6]:
no_segment_flatten_queries_train = []
for query in train_set:
    for rd in query['relevant_docs']:
        no_segment_flatten_queries_train.append({
            'query_id': query['query_id'],
            'query': query['query'],
            'relevant_doc': rd,
        })

In [7]:
def process_relevant_doc_id(relevant_doc_id):
    second_underscore_index = relevant_doc_id.find("_", relevant_doc_id.find("_") + 1)
    if second_underscore_index != -1:
        id_part = relevant_doc_id[:second_underscore_index]
        chunk_part = relevant_doc_id[second_underscore_index + 1:]
        chunk_part = int(chunk_part) if chunk_part.isdigit() else None
        return id_part, chunk_part
    else:
        return relevant_doc_id, None

In [8]:
import re
def chunk_text_with_subheadings(text):
    if re.search(r'\d+\.', text) or re.search(r'[a-z]\)', text, flags=re.IGNORECASE):
        chunks = re.split(r'\n(?=\d+\.)', text)
        final_chunks = [chunk.strip() for chunk in chunks]
    else:
        final_chunks = [text.strip()]
    
    return final_chunks

In [9]:
def get_first_chunk_with_part(list_chunks, chunk_part):
    pattern = re.compile(rf'^\s*{chunk_part}\s*[\.\)\s]')
    
    for chunk in list_chunks:
        if pattern.match(chunk):
            return chunk
    
    return None

In [10]:
no_segment_data_train = []
corpus_dict = {doc['article_id']: doc for doc in no_segment_flatten_corpus}

for query in no_segment_flatten_queries_train:
    relevant_doc_id, chunk_part = process_relevant_doc_id(query['relevant_doc'])
    
    if relevant_doc_id in corpus_dict:
        matched_doc = corpus_dict[relevant_doc_id]
        if chunk_part is None:
            content_str = ' '.join(map(str, matched_doc['content']))
            positive = f"{matched_doc['name']}. {content_str}"
            no_segment_data_train.append({
                'query': query['query'],
                'positive': positive
            })
        else:
            list_chunks = chunk_text_with_subheadings(' '.join(map(str, matched_doc['content'])))
            chunk_part_match = get_first_chunk_with_part(list_chunks, chunk_part )
            if chunk_part_match is not None:
                no_segment_data_train.append({
                'query': query['query'],
                'positive': f"{matched_doc['name']}. {chunk_part_match}"
            })

In [11]:
segmented_data_train = []
for query in no_segment_data_train:
    segmented_data_train.append({
        'query' : ' '.join(rdrsegmenter.word_segment(query['query'])),
        'positive' : ' '.join(rdrsegmenter.word_segment(query['positive']))
    })

In [12]:
train_dataset = []
train_dataset = [InputExample(texts = [a['query'], a['positive']]) for a in segmented_data_train]

In [13]:
word_embedding_model = models.Transformer("keepitreal/vietnamese-sbert", max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode = 'mean')

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [14]:
num_epochs = 5
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=32)
train_loss = losses.MultipleNegativesRankingLoss(model=model)
warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1)

In [15]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    optimizer_class=torch.optim.AdamW,
    optimizer_params={
        'lr': 2e-5
    },
    weight_decay=0.01,
    output_path="/kaggle/working/finetuned_vietsbert_5epoch_15k"
)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.133100
1000,0.032700
1500,0.017600


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

# Evalute

In [8]:
import json
import pickle

with open('/kaggle/input/law-train-evalute/chunk_id_mapping', 'rb') as f:
    idx_mapping = pickle.load(f)
with open('/kaggle/input/law-train-evalute/chunk_corpus', 'rb') as f:
    segmented_chunk_corpus = pickle.load(f)
with open('/kaggle/input/law-train-evalute/queries_relevant_chunks', 'rb') as f:
    segmented_queries_relevant_chunk = pickle.load(f)
    
query_test_set = test_set

In [9]:
query_test_relevant_chunks = []

for query in query_test_set:
    
    relevant_chunks = []
    
    for relevant_doc in query['relevant_docs']:
        try:
            relevant_chunks += idx_mapping[relevant_doc]
        except KeyError:
            continue
        
    query_test_relevant_chunks.append({
        'query_id' : query['query_id'],
        'query' : query['query'],
        'relevant_docs' : query['relevant_docs'],
        'relevant_chunks' : relevant_chunks
    })

In [10]:
segmented_query_test_relevant_chunks = []

for query in query_test_relevant_chunks:
    segmented_query_test_relevant_chunks.append({
        'query_id' : query['query_id'],
        'query' : ''.join(rdrsegmenter.word_segment(query['query'])),
        'relevant_docs' : query['relevant_docs'],
        'relevant_chunks' : query['relevant_chunks']
    })

In [11]:
query_test_relevant_chunks[0]

{'query_id': '66635708c2f544363eed5946',
 'query': 'Ai có thẩm quyền chủ trì cuộc họp Hội đồng thành viên doanh nghiệp nhà nước?',
 'relevant_docs': ['59/2020/qh14_98_2'],
 'relevant_chunks': []}

In [19]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

corpus = {}
queries = {}
relevant_docs = {}
corpus_idx = 0
query_idx = 0

for query in segmented_query_test_relevant_chunks:
    queries[query['query_id']] = query['query']
    relevant_docs[query['query_id']] = set(query['relevant_chunks'])

for doc in segmented_chunk_corpus:
    corpus[doc['chunk_id']] = doc['text']

## Evalute 15k-5 epochs

In [20]:
IR_evaluator = InformationRetrievalEvaluator(queries,corpus,relevant_docs)
model.evaluate(IR_evaluator, output_path="/kaggle/working/finetuned_vietsbert_5epoch_15k_chunk")

{'cosine_accuracy@1': 0.5339645944833264,
 'cosine_accuracy@3': 0.7657472210786331,
 'cosine_accuracy@5': 0.8184438040345822,
 'cosine_accuracy@10': 0.8752573075339646,
 'cosine_precision@1': 0.5339645944833264,
 'cosine_precision@3': 0.42884588994099077,
 'cosine_precision@5': 0.34491560312885966,
 'cosine_precision@10': 0.2276657060518732,
 'cosine_recall@1': 0.18021141393105164,
 'cosine_recall@3': 0.41617964564444593,
 'cosine_recall@5': 0.5302289745918812,
 'cosine_recall@10': 0.6661869864350309,
 'cosine_ndcg@10': 0.5790874359367125,
 'cosine_mrr@10': 0.6569157730335166,
 'cosine_map@100': 0.5085814021115296,
 'dot_accuracy@1': 0.5183202964182791,
 'dot_accuracy@3': 0.7406340057636888,
 'dot_accuracy@5': 0.8065047344586249,
 'dot_accuracy@10': 0.8674351585014409,
 'dot_precision@1': 0.5183202964182791,
 'dot_precision@3': 0.4141622066694113,
 'dot_precision@5': 0.33808151502676,
 'dot_precision@10': 0.22363112391930837,
 'dot_recall@1': 0.17321119353025488,
 'dot_recall@3': 0.398

In [21]:
import pandas as pd
df = pd.read_csv('/kaggle/working/finetuned_vietsbert_5epoch_15k_chunk/Information-Retrieval_evaluation_results.csv')
df

,epoch,steps,cosine-Accuracy@1,cosine-Accuracy@3,cosine-Accuracy@5,cosine-Accuracy@10,cosine-Precision@1,cosine-Recall@1,cosine-Precision@3,cosine-Recall@3,...,dot-Recall@1,dot-Precision@3,dot-Recall@3,dot-Precision@5,dot-Recall@5,dot-Precision@10,dot-Recall@10,dot-MRR@10,dot-NDCG@10,dot-MAP@100
0,-1,-1,0.533965,0.765747,0.818444,0.875257,0.533965,0.180211,0.428846,0.41618,...,0.173211,0.414162,0.398022,0.338082,0.518876,0.223631,0.652884,0.640681,0.563689,0.492671
